# Treino completo

In [ ]:
import glob
import os
from datetime import datetime

import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.optim as optim
from model import DGCNN
from sklearn import preprocessing
from utils import generating_data
from glob import glob

lr_dgcnn = 0.01
weight_decay_dgcnn = 5e-4 #tem no codigo do GMSS tmbm
K=2
epochs=20 # DEFFERARD TMBM USA 20
device = 'cpu'

hiperparam_run = {
    'lr_dgcnn ':lr_dgcnn,
    'weight_decay_dgcnn':weight_decay_dgcnn,
    'K':K,
    'epochs':epochs
}

label_list = np.array([1,0,-1,-1,0,1,-1,0,1,1,0,-1,0,1,-1]) + 1 # -1 for 
fn = 'de_LDS'


full_path = "/home/joaovmrf/Documents/mestrado/SEED/SEED_EEG/SEED_EEG/ExtractedFeatures"
all_files = [each_file for each_file in glob(full_path+'/*.mat') if 'label.mat' not in each_file]
all_files.sort()
# all_files = glob.glob(full_path+"*")

all_acc_results = []
for cur_file in all_files: 
    #item = "15_20130709.mat"
    #item = "13_20140527.mat"
    item = os.path.basename(cur_file)
    cur_user = item.split("_")[0]
    user_session = datetime.strptime(item.split("_")[1][:-4],"%Y%m%d")
    print(f"- reading file: {item}\n--user:{cur_user}, sessions: {user_session}")

    # cur_file = os.path.join(full_path, item)
    all_data = sio.loadmat(cur_file)


    train_data, test_data, train_label, test_label = generating_data(all_data, label_list, fn)

    #train_data = preprocessing.scale(train_data)
    #test_data = preprocessing.scale(test_data)

    train_data_tensor = torch.from_numpy(train_data).to(torch.float)#.to(device)
    test_data_tensor = torch.from_numpy(test_data).to(torch.float)#.to(device)
    train_label_tensor = torch.from_numpy(train_label).to(torch.long)#.to(device)
    test_label_tensor = torch.from_numpy(test_label).to(torch.long)#.to(device)


    # define model
    model = DGCNN(
        in_channels=5, #5 frequencias
        num_electrodes=62, #62 eletrodos
        k_adj=K, #numero de layers - copiando do GMSS - na verdade no GMSS K=ordem de chebyshev
        # out_channels=32, #copiando do GMSS, mas ainda nao entendi
        out_channels=62,
        num_classes=3
    )

    best_test_res = {
        'acc':0,
        'predict_label':None,
        'trur_label':None
    }

    optimizer = optim.Adam(model.parameters(),
                        lr=lr_dgcnn, weight_decay=weight_decay_dgcnn)
    myloss = nn.CrossEntropyLoss()

    for cur_epoch in range(epochs):
        # ordem 1
        # model.train() # como apontar  referencia aos dados de treino? R: E na definicao do modelo
        # optimizer.zero_grad() #etapa comum em todos, zerar o gradiente.

        # output = model(train_data_tensor)
        # #TODO accuracy.
        # #print(output)

        # # import torch.nn.functional as F
        # # loss_train = F.nll_loss(output, train_label)

        # myloss = nn.CrossEntropyLoss()
        # loss = myloss(output, train_label_tensor)
        # loss.backward()
        # optimizer.step()

        # ordem 2
        model.train() # como apontar  referencia aos dados de treino? R: E na definicao do modelo
        output = model(train_data_tensor)
        # STEP 10 - Calculate the loss function
        loss = myloss(output, train_label_tensor)
        
        optimizer.zero_grad() #etapa comum em todos, zerar o gradiente.

        #TODO accuracy.
        #print(output)

        # import torch.nn.functional as F
        # loss_train = F.nll_loss(output, train_label)
        # STEP 11 - Updating the Adj matrix
        loss.backward()
        optimizer.step()
        # Estou sentindo falta do coidgo de atualizacao dessa matriz

        train_acc = torch.sum(torch.argmax(output, 1) == train_label_tensor) / train_data_tensor.shape[0]

        print('Epoch : {} -- TrainLoss : {} -- TrainAcc : {}\n'.format(cur_epoch, loss.cpu().data, train_acc.cpu().data))

    model.eval()
    test_output = model(test_data_tensor)
    test_loss = myloss(test_output, test_label_tensor)
    test_acc = torch.sum(torch.argmax(test_output, 1) == test_label_tensor) / test_data_tensor.shape[0]
    print('Epoch : {} -- TestLoss : {} -- TestAcc : {}\n'.format(cur_epoch, test_loss.cpu().data, test_acc.cpu().data))

    # save the best results    
    # if best_test_res['acc'] < test_acc.cpu().data:
    best_test_res['acc'] = test_acc.cpu().data 
    best_test_res['loss'] = test_loss.cpu().data 
    best_test_res['predict_label'] = torch.argmax(test_output, 0).cpu().numpy()
    best_test_res['true_label'] = test_label_tensor.cpu().numpy()
    print('update res')

    all_acc_results.append([item,best_test_res])

    print(best_test_res)



In [ ]:
all_acc_results

In [ ]:
import pandas as pd

In [ ]:
gcnn_result_df = pd.DataFrame(all_acc_results)
gcnn_result_df

In [ ]:
gcnn_result_df[["user", "session"]]  = pd.DataFrame(gcnn_result_df[0].str.split('_').to_list())
gcnn_result_df[['acc','predict_label','trur_label','loss','true_label']] = pd.DataFrame(gcnn_result_df[1].to_list())
gcnn_result_df

In [ ]:
gcnn_result_df['acc'] = gcnn_result_df['acc'].apply(lambda x: x.item())

In [ ]:
# eda_gcnn = gcnn_result_df[['user', 'session','acc', 'predict_label','trur_label','loss','true_label']].sort_values(['user', 'acc'], ascending=[False, False])
eda_gcnn = gcnn_result_df[['user', 'session','acc']].sort_values(['user', 'acc'], ascending=[False, False])

eda_gcnn

In [ ]:
eda_gcnn.reset_index().to_csv(f"GCNN_WITH_EVAL.csv")


In [ ]:
eda_gcnn.groupby(['user', 'session']).max()#.acc.mean()

In [ ]:
eda_gcnn.groupby(['user']).head(2).reset_index(drop=True)#.acc.mean()

# analisar dataset

In [1]:
import glob
import os
from datetime import datetime

import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.optim as optim
from model import DGCNN
from sklearn import preprocessing
from utils import generating_data
from glob import glob

/home/joaovmrf/miniconda3/envs/ppget_env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
fn = 'de_LDS'

label_list = np.array([1,0,-1,-1,0,1,-1,0,1,1,0,-1,0,1,-1]) + 1 # -1 for 

full_path = "/home/joaovmrf/Documents/mestrado/SEED/SEED_EEG/SEED_EEG/ExtractedFeatures"
all_files = [each_file for each_file in glob(full_path+'/*.mat') if 'label.mat' not in each_file]
all_files.sort()
# all_files = glob.glob(full_path+"*")


# for cur_file in all_files: 
#item = "15_20130709.mat"
#item = "13_20140527.mat"
cur_file = all_files[10]
item = os.path.basename(cur_file)
cur_user = item.split("_")[0]
user_session = datetime.strptime(item.split("_")[1][:-4],"%Y%m%d")
print(f"- reading file: {item}\n--user:{cur_user}, sessions: {user_session}")

# cur_file = os.path.join(full_path, item)
all_data = sio.loadmat(cur_file)


train_data, test_data, train_label, test_label = generating_data(all_data, label_list, fn)

- reading file: 13_20140603.mat
--user:13, sessions: 2014-06-03 00:00:00


	### TRAIN ###
		de_LDS1 - video 1 - (235, 62, 5) observacoes
		de_LDS2 - video 2 - (233, 62, 5) observacoes
		de_LDS3 - video 3 - (206, 62, 5) observacoes
		de_LDS4 - video 4 - (238, 62, 5) observacoes
		de_LDS5 - video 5 - (185, 62, 5) observacoes
		de_LDS6 - video 6 - (195, 62, 5) observacoes
		de_LDS7 - video 7 - (237, 62, 5) observacoes
		de_LDS8 - video 8 - (216, 62, 5) observacoes
		de_LDS9 - video 9 - (265, 62, 5) observacoes
	### TEST ###
		de_LDS10 - video 10 - (237, 62, 5) observacoes
		de_LDS11 - video 11 - (235, 62, 5) observacoes
		de_LDS12 - video 12 - (233, 62, 5) observacoes
		de_LDS13 - video 13 - (235, 62, 5) observacoes
		de_LDS14 - video 14 - (238, 62, 5) observacoes
		de_LDS15 - video 15 - (206, 62, 5) observacoes


In [18]:
test_data.size

429040

In [17]:
test_data_tensor = torch.from_numpy(test_data).to(torch.float)#.to(device)
test_data_tensor.size()

torch.Size([1384, 62, 5])

In [46]:
# pegando apenas o primeiro video (236), todos os canais, a ultima frequencia
test_data_tensor[:236,:,4]

tensor([[18.6900, 17.7714, 18.8810,  ..., 18.0466, 18.1342, 18.1445],
        [18.6895, 17.7718, 18.8819,  ..., 18.0443, 18.1328, 18.1431],
        [18.6886, 17.7711, 18.8821,  ..., 18.0420, 18.1315, 18.1419],
        ...,
        [18.3036, 17.4334, 18.2986,  ..., 18.3583, 18.2486, 18.2527],
        [18.3007, 17.4315, 18.2951,  ..., 18.3586, 18.2484, 18.2525],
        [18.2992, 17.4308, 18.2932,  ..., 18.3590, 18.2481, 18.2521]])

In [51]:
# pegando apenas o primeiro video (236), canal 1, frequencia delta
test_data_tensor[:236,0,0].size()

torch.Size([236])

# Mudando eixo para entneder matriz de features

In [54]:
test_data.shape

(1384, 62, 5)

In [59]:
test_data_swapped = np.swapaxes(test_data, 0, 1)
test_data_swapped = np.swapaxes(test_data_swapped, 1, 2)
test_data_swapped.shape

(62, 5, 1384)

In [60]:
test_data_swapped_tensor = torch.from_numpy(test_data_swapped).to(torch.float)#.to(device)


In [63]:
test_data_swapped_tensor.size()

torch.Size([62, 5, 1384])

In [66]:
# pegando todos os 62 canais, apenas frequencia delta 
test_data_swapped_tensor[:,0,:].size()

torch.Size([62, 1384])

429040